In [1]:
# ok core problem here.. i want to know if TF is faster in channel first or channel last mode

In [1]:
import numpy as np

In [2]:
import keras 
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from keras.layers.core import Dense, Dropout, Activation, Flatten

from keras.constraints import maxnorm

import libs.metrics

Using TensorFlow backend.


In [3]:
# create random data channel_last
idata = np.random.random((20000,100,100,3))
idata2 = np.random.random((20000,3,100,100))
odata = (np.random.random((20000,3)) > 0.5) * 1.0

In [4]:
print idata.shape, idata2.shape

(20000, 100, 100, 3) (20000, 3, 100, 100)


In [3]:
# note due to the data shape.. 
keras.backend.set_image_data_format('channels_last')
print keras.backend.image_data_format()

channels_last


In [7]:
inputs = Input((100, 100, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

flat = Flatten()(pool1)     
output = Dense(3,activation='sigmoid')(flat)

model = Model(inputs=inputs, outputs=output)
model.compile(optimizer=Adam(),
    loss='binary_crossentropy', 
    metrics=['accuracy',
        libs.metrics.f1_score, 
        libs.metrics.precision, 
        libs.metrics.recall])

print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 80000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 240003    
Total params: 250,147
Trainable params: 250,147
Non-trainable params: 0
_________________________________________________________________
None

In [10]:
%%timeit
hist = model.fit(idata, odata, batch_size=64, epochs=10, verbose=False)

1 loop, best of 3: 4min 48s per loop


In [8]:
# note due to the data shape.. 
keras.backend.set_image_data_format('channels_first')
print keras.backend.image_data_format()

channels_first


In [9]:
inputs = Input((3, 100, 100))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

flat = Flatten()(pool1)     
output = Dense(3,activation='sigmoid')(flat)

model2 = Model(inputs=inputs, outputs=output)
model2.compile(optimizer=Adam(),
    loss='binary_crossentropy', 
    metrics=['accuracy',
        libs.metrics.f1_score, 
        libs.metrics.precision, 
        libs.metrics.recall])

print model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3, 100, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 100, 100)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 100, 100)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 50, 50)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 80000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 240003    
Total params: 250,147
Trainable params: 250,147
Non-trainable params: 0
_________________________________________________________________
None

In [11]:
%%timeit
hist2 = model2.fit(idata2, odata, batch_size=64, epochs=10, verbose=False)

1 loop, best of 3: 17min 44s per loop


In [8]:
# note due to the data shape.. 
keras.backend.set_image_data_format('channels_last')
print keras.backend.image_data_format()

channels_last


In [9]:
inputs = Input((100, 100, 3))
x = inputs

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)     
output = Dense(3,activation='sigmoid')(x)

model3 = Model(inputs=inputs, outputs=output)
model3.compile(optimizer=Adam(),
    loss='binary_crossentropy', 
    metrics=['accuracy',
        libs.metrics.f1_score, 
        libs.metrics.precision, 
        libs.metrics.recall])

print model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 21, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 10, 32)        0         
__________

In [10]:
%%timeit
model3.fit(idata, odata, batch_size=64, epochs=10, verbose=False)

1 loop, best of 3: 2min 34s per loop


In [5]:
# note due to the data shape.. 
keras.backend.set_image_data_format('channels_first')
print keras.backend.image_data_format()

channels_first


In [6]:
inputs = Input((3, 100, 100))
x = inputs

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)     
output = Dense(3,activation='sigmoid')(x)

model4 = Model(inputs=inputs, outputs=output)
model4.compile(optimizer=Adam(),
    loss='binary_crossentropy', 
    metrics=['accuracy',
        libs.metrics.f1_score, 
        libs.metrics.precision, 
        libs.metrics.recall])

print model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 100, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 98, 98)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 49, 49)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 47, 47)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 23, 23)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 21, 21)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 10, 10)        0         
__________

In [7]:
%%timeit
model4.fit(idata2, odata, batch_size=64, epochs=10, verbose=False)

1 loop, best of 3: 11min 8s per loop
